In [1]:
import time
import pickle
import numpy as np
import os
import re
import pandas as pd

from Scraping_Library import get_style_links_and_nums, get_beer_info, get_brew_beer_links, get_style_max, open_new_style_page

In [2]:
os.chdir('../../Beergression_Data_and_misc/Data/')

In [3]:
def compile_beer_info():
    
    """
    No inputs required.
    
    Obtains all style links from the style links function.
    
    For each style link, finds the style max (i.e. the last page of beers for that style) and sets a counter to 0.
    
    For each style link combined with the counter; obtains a list of urls of all beer pages on the respective style page, scrapes each page, then increase the counter by 50 so as to go to the next style page, obtain a list of beer links, and scrape each beer page.
    
    The function will then save the dictionary as a pandas DataFrame.
    
    """
    
    style_links = get_style_links()
    
    for style_link in style_links:
        
        style_max = get_style_max(style_link)
        
        print(style_max)

        # For each style link we add the suffix used
        # to sort. Sorting is done in groups of 50.
        # Our counter is used to specify where sorting
        # will occur on the next loaded page.

        # Each style like will have multple pages.

        counter = 0

        while counter <= style_max:
            
            dic = {}

            url = open_new_style_page(style_link,counter)

            brew_beer_links = get_brew_beer_links(url)

            for brew_beer_link in brew_beer_links:

                key, values = get_beer_info(brew_beer_link)

                dic[key] = values

                time.sleep(np.random.poisson(10)/100)

            string = re.findall(r'\d+', url)
            name = str(string[0]) + '_' + str(string[-1])

            df = pd.DataFrame(dic)     
            df = df.transpose()      
            pd.DataFrame.to_pickle(df,name)

            counter += 50
    
    return 

In [6]:
def compile_beer_info():
    
    style_links, style_nums = get_style_links_and_nums()
    
    i = 1
    
    for style_link, style_num in zip(style_links, style_nums):
        
        df = pd.DataFrame(columns = ['beer_name','brewery_name','abv',
                             'ratings','score','brewery_nums'])
        
        style_max = get_style_max(style_link)
        
        counter = 0

        while counter <= style_max:        

            url = open_new_style_page(style_link,counter)

            beer_info_df = get_beer_info(url)
            
            df = pd.concat((df,beer_info_df),axis=0)
            
            time.sleep(np.random.poisson(10)/100)
            
            counter += 50
        
        print('completed pass ' + str(i))
        
        i += 1
        
        pd.DataFrame.to_pickle(df,style_num)

In [7]:
%%time
compile_beer_info()

ValueError: could not convert string to float: 'No create an account now.\n\nYes my password is:\n\n\nForgot your password?'